In [0]:
!wget http://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip
!unzip stanford-corenlp-full-2018-10-05.zip
!pip install stanfordcorenlp
!pip install https://github.com/huggingface/neuralcoref-models/releases/download/en_coref_sm-3.0.0/en_coref_sm-3.0.0.tar.gz
!pip install -y spacy==2.0.12
!pip install cymem==1.31.2
!pip install neuralcoref --no-binary neuralcoref
!pip install word2number

In [0]:
!pip install textacy

In [0]:
import pandas as pd

import en_coref_sm
from stanfordcorenlp import StanfordCoreNLP
from nltk.parse.corenlp import CoreNLPParser
import visualization

STANFORD_CORENLP_PATH = 'stanford-corenlp-full-2018-10-05/'
PORT = 9090
try:
    server = StanfordCoreNLP(STANFORD_CORENLP_PATH, port=PORT, quiet=True)
except OSError as e:
    print('The port is occupied, probably an instance is already running.')
    server = StanfordCoreNLP('http://localhost', port=PORT, quiet=True)
    
STANFORD_SERVER_URL = server.url

In [0]:
# create model 
import spacy
import en_coref_sm

stanford_model = CoreNLPParser(url=STANFORD_SERVER_URL)

huggingface_model = spacy.load('en_coref_sm')

try:
    stanford_model.api_call('This is a dummy text.', properties={'annotators': 'coref'})
except:
    pass

In [0]:
#!python -m spacy download en_core_web_sm

In [0]:
from spacy.attrs import IS_TITLE
import time
from word2number import w2n                    
import spacy
nlp = spacy.load('en_core_web_sm')

import pandas as pd

df = pd.read_csv('csv_articles')


article_=[]
#sent_articles_=[]
POS_ = []
    

# Seek of tags

def find_pos(POS_, tag_):
    
    return [x for i, x in enumerate(POS_) if POS_[i][3]==';']
  
#Transform to sentenses

def transform_text(df_):
    sent_articles_=[]
    df=df_.dropna()
    for j in df.index:
        doc = nlp(df.text[j])
        #print(j)
        for sent in doc.sents: sent_articles_.append(sent)
    return sent_articles_

#create_POS  
def create_POS (df):
    df=df.dropna()
    for j in df.index:
        doc = nlp(df.text[j])
        #print(j)
        for i, sent in enumerate(doc.sents): 
            sent_articles_.append(sent)
            for token in sent:
                POS_.append((j, i, token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop))  
    return POS_ 

In [0]:
#Transform words to numerics
def num_sent(doc_):

    row_=''
    text=''
    first=doc_[0]
    buffer1_=[]
        
    for token in doc_:
        try:
            if (token.tag_=='CD') and (first.tag_=='CD'):
                  if len(row_)<3:
                     row_ += str(first.lemma_) +' '+ str(token.lemma_)+' '
                  else:
                     row_ += str(token.lemma_) + ' ' 
                  
                  #buffer_=[]
                  
                  first= token
                  #text += str(w2n.word_to_num(row_)) +' '
                  
            elif (token.tag_=='CD'):
                      try: 
                          buffer1_.append(token.lemma_)
                          first= token
                          #text += str(w2n.word_to_num(token.lemma_)) +' '
                           
                      except ValueError:
                          
                          first= token 
                          text += token.text +' '
            else: 
                if len(buffer1_)>0 and token.tag_!='CD' and len(row_)>0:
                    text += str(w2n.word_to_num(row_)) +' '+ token.text+' '
                    buffer1_=[]
                    
                    row_= ''
                elif (len(buffer1_)>0) and (token.tag_!='CD')  and (len(row_) == 0): 
                    text += str(w2n.word_to_num(buffer1_[0])) +' '+ token.text+' '
                    first= token
                    buffer1_=[]
                    
                else:
                    first= token
                    text += token.text +' '
        except ValueError:
                    
                    text += token.text +' '
    return text

#Pipeline for text with transform words to numerics

def pipe_(df,i):
    #for example  df[0:1]
    df_=df[0:i]
    #print(df_)
    articles_= transform_text(df_)
    print(articles_)
    for i in range(0,articles_.__len__()):
        #sent_articles_[i][1]
        doc_=nlp(str(articles_[i]))
        print(i)
        try: 
            print(num_sent(doc_))
        except ValueError:
            print(doc_)
            
def depend_(sent):
  doc=nlp(sent)
  sub_toks = [tok for tok in doc if (tok.dep_ == "nsubj") ]
  verb_toks = [tok for tok in doc if (tok.dep_ == "ROOT") ]
  obj_toks = [tok for tok in doc if (tok.dep_ == "dobj") ]
  print(sub_toks) 
  print(verb_toks) 
  print(obj_toks)             

In [0]:
from IPython.core.display import display, HTML
# Add css styles and js 
display(HTML(open('visualization/highlight.css').read()))
display(HTML(open('visualization/highlight.js').read()))

def out_coref(row,i):
    try:
       data = stanford_model.api_call(row, properties={'annotators': 'coref'})
       html = visualization.render(data, stanford=True, jupyter=False)
       rows.append({'sample_idx': i, 
                 'model': 'Stanford',
                 'annotation': html})
    
     
       data = huggingface_model(row)
       html = visualization.render(data, huggingface=True, jupyter=False)
       rows.append({'sample_idx': i, 'model': 'NeuroCoref',
                 'annotation': html})
            
        
    except:
       print('Not text')
       
        
        
rows = []

#Get first sentense from df df[0:1]

sent_articles_=transform_text(df[0:2])

for i in range(len(sent_articles_)):
    row =str(sent_articles_[i])
    out_coref(row,i)
    
    
df_all = pd.DataFrame(rows).groupby(['sample_idx','model']).agg(lambda x: x)
s = df_all.style.set_properties(**{'text-align': 'left'})

display(HTML(s.render(justify='left')))

In [0]:
pipe_(df,1)

# **Identify sentance subject. Identify object. Identify predicate. Determine and tag if object is Direct, Indirect, or Object of a Preposition •Identify adjectives and adverbs, and mark which nouns and verbs**

In [0]:
def depend_(sent): # Identify sentance subject. Identify object. Identify predicate
  doc=nlp(sent)
  sub_toks = [tok for tok in doc if (tok.dep_ == "nsubj") ]
  verb_toks = [tok for tok in doc if (tok.dep_ == "ROOT") ]
  obj_toks = [tok for tok in doc if (tok.dep_ == "dobj") ]
  print("Subject: ", sub_toks) 
  print("ROOT: ", verb_toks) 
  print("Object: ", obj_toks) 

In [0]:
print(sent_articles_[0])

depend_(str(sent_articles_[0]))

In [0]:
from __future__ import unicode_literals
# Load Library files
import textacy
nlp = spacy.load('en_core_web_sm')
SUBJ = ["nsubj","nsubjpass"] 
VERB = ["ROOT"] 
OBJ = ["dobj", "pobj", "dobj"] 
text = nlp(str(sent_articles_[0]))
sub_toks = [tok for tok in text if (tok.dep_ in SUBJ) ]
obj_toks = [tok for tok in text if (tok.dep_ in OBJ) ]
vrb_toks = [tok for tok in text if (tok.dep_ in VERB) ]
text_ext = list(textacy.extract.subject_verb_object_triples(text))

print(sent_articles_[0])
print("Subjects:", sub_toks)
print("VERB :", vrb_toks)
print("OBJECT(s):", obj_toks)
print ("Extract SUB-VERB-OBJ:", text_ext)